In [ ]:
!pip install dash
!pip install plotly
!pip install pandas
!pip install numpy
!pip install pyinstaller


In [8]:
from dash import Dash, dcc, html, Input, Output, State, dash_table
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import base64
import io
import smtplib
from datetime import datetime
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os

# Inicialização do aplicativo Dash
app = Dash(__name__)

# Obtendo o ano atual
ano_atual = datetime.now().year

# Lista de opções para ano (iniciando do ano atual e indo para os anteriores)
anos = [{'label': str(year), 'value': year} for year in range(ano_atual, 1999, -1)]

# Lista de opções para média de aprovação
medias_aprovacao = [{'label': str(media), 'value': media} for media in range(1, 11)]

# Layout do aplicativo
app.layout = html.Div([
    html.H1("Análise de Notas dos Estudantes", style={'text-align': 'center', 'color': 'white'}),
    
    # Informações gerais
    html.Div([
        html.Label("Nome do Professor:", style={'color': 'white', 'marginRight': '10px'}),
        dcc.Input(id='professor', type='text', placeholder="Nome do Professor", style={'marginRight': '20px'}),
        html.Label("Instituição:", style={'color': 'white', 'marginRight': '10px'}),
        dcc.Input(id='instituicao', type='text', placeholder="Nome da Instituição", style={'marginRight': '20px'}),
    ], style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center'}),
    
    html.Br(),
    html.Div([
        html.Label("Unidade Curricular:", style={'color': 'white', 'marginRight': '10px'}),
        dcc.Input(id='disciplina', type='text', placeholder="Nome da Disciplina", style={'marginRight': '20px'}),
        html.Label("Turma:", style={'color': 'white', 'marginRight': '10px'}),
        dcc.Input(id='turma', type='text', placeholder="Nome da Turma (Opcional)", style={'marginRight': '20px'}),
        html.Label("Semestre:", style={'color': 'white', 'marginRight': '10px'}),
        dcc.Input(id='semestre', type='text', placeholder="Semestre", style={'marginRight': '20px'}),
        html.Label("Ano:", style={'color': 'white', 'marginRight': '10px'}),
        dcc.Dropdown(id='ano', options=anos, placeholder="Ano", value=ano_atual, style={'width': '100px'}),
    ], style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center'}),
    
    html.Br(),
    
    # Seleção de média de aprovação
    html.Div([
        html.Label("Média para Aprovação:", style={'color': 'white', 'marginRight': '10px'}),
        dcc.Dropdown(
            id='media-aprovacao',
            options=medias_aprovacao,
            value=6,  # Valor padrão de média para aprovação
            style={'width': '60px'},
            clearable=False
        ),
    ], style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center'}),
    
    html.Br(),
    
    # Entrada de dados de estudantes
    html.Div([
        html.Label("Insira os nomes dos estudantes e suas notas:", style={'color': 'white', 'marginRight': '10px'}),
        dcc.Input(id='input-nome', type='text', placeholder="Nome", style={'marginRight': '20px', 'width': '150px'}),
        dcc.Input(id='input-nota', type='number', placeholder="Nota", style={'marginRight': '20px', 'width': '80px'}),
        html.Button('Adicionar', id='add-button', n_clicks=0),
    ], style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center'}),
    
    html.Br(),
    html.H3("Tabela de Notas", style={'text-align': 'center', 'color': 'white'}),
    
    dash_table.DataTable(
        id='table',
        columns=[{"name": "Nome", "id": "Nome"}, {"name": "Nota", "id": "Nota"}],
        data=[], 
        editable=True,
        style_header={'backgroundColor': 'rgb(30, 30, 30)', 'color': 'white'},
        style_cell={'backgroundColor': 'rgb(50, 50, 50)', 'color': 'white'},
    ),
    
    html.Br(),
    html.Div([
        html.Label("Ou faça o upload de um arquivo CSV:", style={'color': 'white'}),
        dcc.Upload(
            id='upload-data',
            children=html.Button('Carregar CSV'),
            style={'margin': 'auto'}
        ),
        html.Div(id='output-data-upload')
    ], style={'text-align': 'center'}),
    
    html.Br(),
    html.Div([
        html.Button('Gerar Análise', id='generate-button', n_clicks=0),
    ], style={'text-align': 'center'}),
    
    dcc.Graph(id='graph'),
    html.Div(id='analysis-output', style={'text-align': 'center', 'color': 'white', 'font-size': '16px'}),
    
    html.Br(),
    
    # Botão para salvar gráfico
    html.Div([
        html.Label("Salvar gráfico como:", style={'color': 'white', 'marginRight': '10px'}),
        dcc.Dropdown(
            id='save-format',
            options=[
                {'label': 'PNG', 'value': 'png'},
                {'label': 'PDF', 'value': 'pdf'},
                {'label': 'SVG', 'value': 'svg'},
                {'label': 'JPEG', 'value': 'jpeg'}
            ],
            placeholder='Selecione o formato',
            style={'width': '200px', 'marginRight': '20px'}
        ),
        html.Button('Salvar', id='save-button', n_clicks=0),
    ], style={'text-align': 'center'}),
    
    html.Br(),
    html.Div([
        html.Label("Enviar gráfico por e-mail:", style={'color': 'white', 'marginRight': '10px'}),
        dcc.Input(id='email-input', type='text', placeholder="E-mails separados por vírgula", style={'width': '300px', 'marginRight': '20px'}),
        html.Button('Enviar', id='send-button', n_clicks=0),
    ], style={'text-align': 'center'}),
    
    # Comparação com outras turmas
    html.H3("Comparação com Outras Turmas", style={'text-align': 'center', 'color': 'white'}),
    html.Label("Selecione a Turma para Comparação:", style={'color': 'white'}),
    dcc.Dropdown(
        id='turma-dropdown',
        options=[],
        placeholder="Selecione a Turma para Comparar",
        style={'width': '50%', 'margin': 'auto'}
    ),
    html.Br(),
    dcc.Graph(id='comparison-graph'),
], style={'backgroundColor': '#333', 'padding': '20px'})

# Função para parsear o arquivo CSV
def parse_contents(contents):
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
    return df

# Callback para adicionar estudantes manualmente
@app.callback(
    Output('table', 'data'),
    [Input('add-button', 'n_clicks')],
    [State('input-nome', 'value'),
     State('input-nota', 'value'),
     State('table', 'data')]
)
def add_student(n_clicks, nome, nota, rows):
    if n_clicks > 0 and nome and nota is not None:
        try:
            nota = float(nota)
            if 0 <= nota <= 10:  # Verificar se a nota está no intervalo correto
                rows.append({'Nome': nome, 'Nota': nota})
        except ValueError:
            pass  # Ignorar entradas inválidas
    return rows

# Callback para ler o arquivo CSV e atualizar a tabela
@app.callback(
    Output('table', 'data', allow_duplicate=True),
    [Input('upload-data', 'contents')],
    [State('table', 'data')],
    prevent_initial_call=True
)
def update_table_from_upload(contents, rows):
    if contents is not None:
        df = parse_contents(contents)
        if 'Nome' in df.columns and 'Nota' in df.columns:
            for index, row in df.iterrows():
                try:
                    rows.append({'Nome': row['Nome'], 'Nota': float(row['Nota'])})
                except ValueError:
                    pass  # Ignorar entradas inválidas no CSV
    return rows

# Callback para gerar o gráfico e a análise
@app.callback(
    [Output('graph', 'figure'), Output('analysis-output', 'children')],
    [Input('generate-button', 'n_clicks')],
    [State('table', 'data'),
     State('professor', 'value'),
     State('instituicao', 'value'),
     State('disciplina', 'value'),
     State('turma', 'value'),
     State('semestre', 'value'),
     State('ano', 'value'),
     State('media-aprovacao', 'value')],
    prevent_initial_call=True
)
def generate_analysis(n_clicks, data, professor, instituicao, disciplina, turma, semestre, ano, media_aprovacao):
    if n_clicks > 0 and data:
        # Verificar se a tabela possui dados válidos
        if len(data) == 0:
            return {}, "Por favor, insira os dados dos alunos para gerar a análise."

        # Converter os dados da tabela para um DataFrame
        df = pd.DataFrame(data)
        try:
            notas = df['Nota'].astype(float).tolist()
        except ValueError:
            return {}, "Erro: Algumas notas não são números válidos."

        # Calcular a média e o desvio padrão
        mu1 = np.mean(notas)
        mu2 = media_aprovacao
        sigma = np.std(notas)

        # Gerar dados para as curvas gaussianas
        x = np.linspace(0, 10, 100)
        y1 = 1/(sigma * np.sqrt(2 * np.pi)) * np.exp(-(x - mu1)**2 / (2 * sigma**2))
        y2 = 1/(sigma * np.sqrt(2 * np.pi)) * np.exp(-(x - mu2)**2 / (2 * sigma**2))

        # Criar gráfico interativo com melhorias
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=y1, mode='lines', name='Resultados da prova', fill='tozeroy', line=dict(color='blue')))
        fig.add_trace(go.Scatter(x=x, y=y2, mode='lines', name='Referência', fill='tozeroy', line=dict(color='orange')))
        fig.add_trace(go.Scatter(x=notas, y=[0]*len(notas), mode='markers+text', text=df['Nome'], textposition="top center", name='Notas dos alunos', marker=dict(color='red', size=10)))
        fig.add_vline(x=mu1, line=dict(color='blue', dash='dash'), annotation_text=f"Média Prova: {mu1:.2f}", annotation_position="top left")
        fig.add_vline(x=mu2, line=dict(color='orange', dash='dash'), annotation_text=f"Média Esperada: {mu2}", annotation_position="top right")
        fig.update_layout(title_text=f"Distribuição das Notas - {disciplina} ({turma})" if turma else f"Distribuição das Notas - {disciplina}",
                          xaxis_title="Notas", yaxis_title="Densidade de Probabilidade",
                          legend_title="Legenda", title_font_size=20,
                          autosize=True, width=1200, height=500)  # Ajuste de tamanho automático
        fig.update_xaxes(range=[0, 10])
        fig.update_yaxes(range=[0, 0.15])

        # Análise do desempenho
        analysis = f"Professor: {professor} | Instituição: {instituicao} | Unidade Curricular: {disciplina} | Turma: {turma if turma else 'Não especificada'} | Semestre: {semestre} | Ano: {ano}.\n"
        if mu1 < mu2:
            analysis += f"A média da prova ({mu1:.2f}) está abaixo da média de aprovação esperada ({mu2}). Considere revisar o conteúdo ou o formato da prova."
        else:
            analysis += f"A média da prova ({mu1:.2f}) está dentro ou acima da média de aprovação esperada ({mu2}). Bom trabalho!"

        return fig, analysis
    return {}, ""

# Callback para salvar o gráfico
@app.callback(
    Output('save-button', 'children'),
    [Input('save-button', 'n_clicks')],
    [State('save-format', 'value'),
     State('graph', 'figure')],
    prevent_initial_call=True
)
def save_graph(n_clicks, format, figure):
    if n_clicks > 0 and format and figure:
        filename = f"grafico_notas_{datetime.now().strftime('%Y%m%d_%H%M%S')}.{format}"
        with open(filename, 'wb') as f:
            f.write(base64.b64decode(figure['data'][0]['base64']))
        return f"Gráfico salvo como {filename}"
    return "Salvar"

# Callback para enviar o gráfico por e-mail
@app.callback(
    Output('send-button', 'children'),
    [Input('send-button', 'n_clicks')],
    [State('email-input', 'value'),
     State('graph', 'figure')],
    prevent_initial_call=True
)
def send_email(n_clicks, email, figure):
    if n_clicks > 0 and email and figure:
        emails = [e.strip() for e in email.split(',')]
        filename = f"grafico_notas_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
        
        # Salvar o gráfico como PNG temporário
        with open(filename, 'wb') as f:
            f.write(base64.b64decode(figure['data'][0]['base64']))
        
        # Configurações de e-mail
        smtp_server = 'smtp.gmail.com'
        smtp_port = 587
        sender_email = 'seuemail@gmail.com'  # Seu e-mail
        sender_password = 'suasenha'         # Sua senha de aplicativo
        
        # Enviar e-mail para cada destinatário
        for recipient in emails:
            msg = MIMEMultipart()
            msg['From'] = sender_email
            msg['To'] = recipient
            msg['Subject'] = "Gráfico de Notas dos Estudantes"

            body = "Segue em anexo o gráfico gerado pela análise de notas dos estudantes."
            msg.attach(MIMEText(body, 'plain'))

            # Anexo
            attachment = open(filename, "rb")
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f"attachment; filename= {filename}")
            msg.attach(part)

            # Enviar o e-mail
            server = smtplib.SMTP(smtp_server, smtp_port)
            server.starttls()
            server.login(sender_email, sender_password)
            text = msg.as_string()
            server.sendmail(sender_email, recipient, text)
            server.quit()

        os.remove(filename)  # Remover arquivo temporário

        return f"Gráfico enviado para {email}"
    return "Enviar"

# Executar o aplicativo
if __name__ == '__main__':
    app.run_server(debug=True)


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[8], line 280, in send_email(
    n_clicks=1,
    email='lucasmoreira115k@gmail.com',
    figure={'data': [{'fill': 'tozeroy', 'line': {'color': 'blue'}, 'mode': 'lines', 'name': 'Resultados da prova', 'type': 'scatter', 'x': [0, 0.10101010101010101, 0.20202020202020202, 0.30303030303030304, 0.40404040404040403, 0.5050505050505051, 0.6060606060606061, 0.7070707070707071, 0.8080808080808081, 0.9090909090909091, 1.0101010101010102, 1.1111111111111112, 1.2121212121212122, 1.3131313131313131, 1.4141414141414141, 1.5151515151515151, 1.6161616161616161, 1.7171717171717171, 1.8181818181818181, 1.9191919191919191, ...], 'y': [0.023694826977726974, 0.025540083016621556, 0.027486940272705523, 0.029536962001567846, 0.031691338425457204, 0.03395085138055632, 0.036315839592730305, 0.03878616495735208, 0.041361180214752644, 0.04403969842497074